# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('../WeatherPy/weather_data.csv')
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hermanus,-34.42,19.23,55.99,94,33,0.92,ZA,1586827199
1,berlevag,70.86,29.09,34.00,89,99,19.57,NO,1586827750
2,bluff,-46.60,168.33,46.99,71,52,20.00,NZ,1586827325
3,kalanguy,51.02,116.52,39.16,52,96,8.32,RU,1586827750
4,rikitea,-23.12,-134.97,76.98,71,79,13.13,PF,1586827324


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
latlon = weather_df[['Lat', 'Lng']]

# Fill NaN values and convert to float
humidity = weather_df["Humidity"]

#latlon.head()
#humidity.head()

In [4]:
# Plot Heatmap
figure_layout = {'width': '900px','height': '600px','border': '1px solid black','padding': '1px','margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))
#fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(latlon, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)
# Add heat layer
fig.add_layer(heat_layer)

# Display map
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
needed_temperature = ((weather_df["Max Temp"] < 80) & (weather_df["Max Temp"] > 70))
needed_windspeed = weather_df["Wind Speed"] < 15
needed_cloudiness = weather_df["Cloudiness"] == 0
needed_criteria = needed_temperature & needed_windspeed & needed_cloudiness 

#drop null values and re-index the df
perfect_weather_df = weather_df[needed_criteria]
perfect_weather_df = perfect_weather_df.dropna()
perfect_weather_df = perfect_weather_df.reset_index()
perfect_weather_df.head(10)

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,66,beloha,-25.17,45.05,73.56,69,0,4.00,MG,1586827757
1,73,marawi,8.00,124.29,78.40,47,0,1.23,PH,1586827758
2,154,jamtara,23.95,86.80,75.76,46,0,2.98,IN,1586827768
3,191,narasannapeta,18.42,84.05,77.27,74,0,4.12,IN,1586827773
4,235,flinders,-34.58,150.86,72.00,43,0,10.29,AU,1586827361
5,262,mayahi,13.96,7.67,79.68,14,0,9.28,NE,1586827782


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = perfect_weather_df
 
hotel_df['Hotel Name'] = ""
hotel_df.head()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,66,beloha,-25.17,45.05,73.56,69,0,4.00,MG,1586827757,
1,73,marawi,8.00,124.29,78.40,47,0,1.23,PH,1586827758,
2,154,jamtara,23.95,86.80,75.76,46,0,2.98,IN,1586827768,
3,191,narasannapeta,18.42,84.05,77.27,74,0,4.12,IN,1586827773,
4,235,flinders,-34.58,150.86,72.00,43,0,10.29,AU,1586827361,


In [7]:
parms = {"radius": 5000,"types": "lodging","keyword": "Hotel","key": g_key}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]

    # loop through new locations
    parms["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=parms)
    
    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        #hotel_df.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        #hotel_df.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df.info()

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 11 columns):
index         6 non-null int64
City          6 non-null object
Lat           6 non-null float64
Lng           6 non-null float64
Max Temp      6 non-null float64
Humidity      6 non-null int64
Cloudiness    6 non-null int64
Wind Speed    6 non-null float64
Country       6 non-null object
Date          6 non-null int64
Hotel Name    6 non-null object
dtypes: float64(4), int64(4), object(3)
memory usage: 656.0+ bytes


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
figure_layout = {'width': '900px','height': '600px','border': '1px solid black','padding': '1px','margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()])

# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…